# データサイエンス100本ノック（構造化データ加工編） - Python

はじめに
初めに以下のセルを実行してください
必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
pandas等、利用が想定されるライブラリは以下セルでインポートしています
その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
処理は複数回に分けても構いません
名前、住所等はダミーデータであり、実在するものではありません

In [42]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [6]:
df_store.head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
5,S13051,板橋大原店,13,東京都,東京都板橋区大原町,トウキョウトイタバシクオオハラチョウ,03-0123-4029,139.6980,35.76788,1045.0
6,S13015,南砂店,13,東京都,東京都江東区南砂二丁目,トウキョウトコウトウクミナミスナニチョウメ,03-0123-4014,139.8229,35.67066,1337.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
8,S13044,南六郷店,13,東京都,東京都大田区南六郷二丁目,トウキョウトオオタクミナミロクゴウニチョウメ,03-0123-4028,139.7207,35.54604,1379.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0


In [7]:
df1 = df_store.query("tel_no.str.contains('^[0-9]{3}.[0-9]{3}.[0-9]{4}',regex=True)",engine='python')
df1

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [9]:
df_customer.dtypes

customer_id             object
customer_name           object
gender_cd                int64
gender                  object
birth_day               object
age                      int64
postal_cd               object
address                 object
application_store_cd    object
application_date         int64
status_cd               object
dtype: object

In [10]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [18]:
#birthdayをdatetime型にする必要がある
df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
df_customer.dtypes

customer_id                     object
customer_name                   object
gender_cd                        int64
gender                          object
birth_day               datetime64[ns]
age                              int64
postal_cd                       object
address                         object
application_store_cd            object
application_date                 int64
status_cd                       object
dtype: object

In [21]:
df_customer.sort_values(by='birth_day',ascending=True).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [22]:
df_customer.sort_values(by='birth_day',ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [43]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [46]:
# #1件当たりのamountを計算?
# df_receipt['amount_per_quantity'] = df_receipt['amount']/df_receipt['quantity']
# df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,amount_per_quantity
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,158.0
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81,81.0
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170,170.0
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25,25.0
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90,90.0


In [51]:
#ランク付け
df_receipt['amount_rank'] = df_receipt['amount'].rank(method='min',ascending=False)
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,amount_per_quantity,amount_rank
1202,20190903,1283472000,S14011,1192,2,CS011415000006,P080104055,1,10925,10925.0,1.0
62317,20190304,1267660800,S14046,1152,2,ZZ000000000000,P090302118,1,6800,6800.0,2.0
54095,20170921,1221955200,S14028,1192,2,CS028605000002,P090907210,1,5780,5780.0,3.0
72747,20190724,1279929600,S13009,1192,2,ZZ000000000000,P090907209,1,5480,5480.0,4.0
10320,20190103,1262476800,S14021,1162,2,ZZ000000000000,P090907208,1,5480,5480.0,4.0


In [52]:
#つけたランクで並び替え
df_receipt = df_receipt.sort_values(by='amount_rank')

In [53]:
#指定の項目を10件表示
df_receipt[['customer_id','amount','amount_rank']].head(10)

,customer_id,amount,amount_rank
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
4632,CS015515000034,5480,4.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,7.0
55795,CS032414000072,5280,9.0
77874,CS008513000063,5280,9.0


P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [55]:
df_receipt['amount_rank'] = df_receipt['amount'].rank(method='first',ascending=False)
df_receipt = df_receipt.sort_values(by='amount_rank')
df_receipt[['customer_id','amount','amount_rank']].head(10)

,customer_id,amount,amount_rank
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,5.0
4632,CS015515000034,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
99190,CS020414000081,5280,9.0
57058,ZZ000000000000,5280,10.0
